In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,32528
2,Huelva,Confirmados PDIA 14 días,1395
3,Huelva,Tasa PDIA 14 días,"271,83974121636106"
4,Huelva,Confirmados PDIA 7 días,587
5,Huelva,Total Confirmados,32709
6,Huelva,Curados,30362
7,Huelva,Fallecidos,388


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  32528.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8848.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 551 personas en los últimos 7 días 

Un positivo PCR cada 235 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,32528.0,587.0,1395.0,513170.0,114.387045,271.839741,46.0
Huelva-Costa,18757.0,369.0,832.0,289548.0,127.440010,287.344413,26.0
Huelva (capital),8848.0,261.0,612.0,143837.0,181.455397,425.481622,19.0
Condado-Campiña,10546.0,169.0,430.0,156231.0,108.173154,275.233468,14.0
Sierra de Huelva-Andévalo Central,2874.0,48.0,128.0,67391.0,71.226128,189.936342,6.0
Moguer,1352.0,39.0,74.0,21867.0,178.350940,338.409475,4.0
Palos de la Frontera,699.0,16.0,56.0,11742.0,136.262988,476.920456,3.0
Aljaraque,1153.0,27.0,66.0,21474.0,125.733445,307.348421,3.0
Cartaya,1365.0,13.0,31.0,20083.0,64.731365,154.359408,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,13.0,5.0,5.0,299.0,1672.240803,1672.240803,0.0
Puebla de Guzmán,132.0,11.0,18.0,3092.0,355.756792,582.147477,0.0
San Juan del Puerto,552.0,28.0,54.0,9411.0,297.524174,573.796621,2.0
Villarrasa,180.0,2.0,12.0,2211.0,90.456807,542.740841,0.0
Valverde del Camino,629.0,22.0,66.0,12750.0,172.549020,517.647059,2.0
Palos de la Frontera,699.0,16.0,56.0,11742.0,136.262988,476.920456,3.0
Huelva (capital),8848.0,261.0,612.0,143837.0,181.455397,425.481622,19.0
Almendro (El),39.0,3.0,3.0,840.0,357.142857,357.142857,1.0
Moguer,1352.0,39.0,74.0,21867.0,178.350940,338.409475,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,128.0,1.0,7.0,3812.0,26.232949,183.630640,0.0,0.142857
Villarrasa,180.0,2.0,12.0,2211.0,90.456807,542.740841,0.0,0.166667
Manzanilla,103.0,1.0,5.0,2118.0,47.214353,236.071766,0.0,0.200000
Villanueva de los Castillejos,164.0,1.0,4.0,2825.0,35.398230,141.592920,0.0,0.250000
Punta Umbría,840.0,9.0,34.0,15355.0,58.612830,221.426246,0.0,0.264706
Palos de la Frontera,699.0,16.0,56.0,11742.0,136.262988,476.920456,3.0,0.285714
Bonares,288.0,5.0,17.0,6060.0,82.508251,280.528053,0.0,0.294118
Trigueros,325.0,7.0,23.0,7862.0,89.035869,292.546426,1.0,0.304348
Bollullos Par del Condado,1015.0,13.0,42.0,14387.0,90.359352,291.930215,0.0,0.309524
